In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import time
from google.oauth2 import service_account
from googleapiclient.discovery import build

load_dotenv()

gtm_id = os.environ["GTM_ACCOUNT_ID"]

# Excel

In [2]:
xlsx_names = ['kyobo', 'library', 'scholar', 'story', 'tree']
service_names = {
    '교보문고_renewal': 'kyobo',
    '전자도서관': 'library',
    '스콜라': 'scholar',
    '스토리': 'story',
    '리딩트리': 'tree'
}

def get_xlsx():
    res = {}
    for name in service_names.values():
        res[name]=pd.ExcelFile(f'./xlsx/{name}.xlsx')

    return res

def get_sheet_name(excel):
    sheet_names = excel.sheet_names
    return sheet_names

def read_sheet(excel_file, sheet_name):
    return pd.read_excel(excel_file, sheet_name=sheet_name)

# GTM

In [3]:
# Get credentials
def get_credentials():
    SERVICE_ACCOUNT_FILE = './xlsx/gtm-api-409502-6f8362f190d7.json'

    credentials_container = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE,
        scopes=["https://www.googleapis.com/auth/tagmanager.edit.containers"]
    )

    return credentials_container

credentials = get_credentials()

# Get service
def get_service():
    service = build('tagmanager', 'v2', credentials = credentials)

    return service

service = get_service()

# Get container
def get_container():
    parent = f"accounts/{gtm_id}"

    containers = service.accounts().containers().list(
        parent=parent,
    ).execute()

    return containers['container']

container_origin = get_container()
containers = {}
for i in container_origin:
    containers[i['name']] = i


# Set Container

In [46]:
# 교보문고_renewal, 전자도서관, 스콜라, 스토리, 리딩트리
service_name = '리딩트리'
container = containers[service_name]

xlsxs = get_xlsx()
xlsx = xlsxs[service_names[service_name]]

# Get workspace
def get_workspaces():
    parent=container['path']

    workspace = service.accounts().containers().workspaces().list(
        parent=parent
    ).execute()

    return workspace['workspace']

workspace_origin = get_workspaces()

workspaces = {}
for i in workspace_origin:
    workspaces[i['name']] = i

workspace = workspaces['new']

# Get Elements

In [47]:
def get_trigger_list():
    parent=workspace['path']

    try:
        response = service.accounts().containers().workspaces().triggers().list(
            parent=parent
        ).execute()

        print(f"Listed triggers: {response['trigger']}")
        return response
    except Exception as e:
        print(f"An error occurred: {e}")
        return e

In [48]:
triggers = get_trigger_list()

trigger_obj = {}

for i in triggers['trigger']:
    trigger_obj[i['name'].lower()] = i['triggerId']


Listed triggers: [{'path': 'accounts/6003640738/containers/171541415/workspaces/5/triggers/90', 'accountId': '6003640738', 'containerId': '171541415', 'workspaceId': '5', 'triggerId': '90', 'name': 'Event - login', 'type': 'customEvent', 'customEventFilter': [{'type': 'equals', 'parameter': [{'type': 'template', 'key': 'arg0', 'value': '{{_event}}'}, {'type': 'template', 'key': 'arg1', 'value': 'login'}]}], 'filter': [{'type': 'equals', 'parameter': [{'type': 'template', 'key': 'arg0', 'value': '{{DLV - platform}}'}, {'type': 'template', 'key': 'arg1', 'value': 'app'}, {'type': 'boolean', 'key': 'negate', 'value': 'true'}]}], 'fingerprint': '1704677314997', 'tagManagerUrl': 'https://tagmanager.google.com/#/container/accounts/6003640738/containers/171541415/workspaces/5/triggers/90?apiLink=trigger'}, {'path': 'accounts/6003640738/containers/171541415/workspaces/5/triggers/91', 'accountId': '6003640738', 'containerId': '171541415', 'workspaceId': '5', 'triggerId': '91', 'name': 'Event - 

# Triggers

In [49]:
event_type_map = {
    '맞춤 이벤트': 'customEvent',
    '페이지뷰': 'pageview',
    '스크롤 깊이': 'scrollDepth',
    '클릭 - 링크만': 'linkClick',
    '클릭만 - 링크': 'linkClick',
    '클릭 - 모든 요소': 'click'
}

filter_type_map = {
    '다음과 같지 않음': 'equals',
    '같음': 'equals',   
    '_x0008_같음': 'equals',
    '포함': 'contains',
    '정규 표현식과 일치': 'matchRegex',
    "정규 표현식과 일치 ": 'matchRegex',
    '다음으로 시작 하지 않음': 'startsWith',
    '다음으로 시작': 'startsWith',
    'CSS 선택 도구와 일치': 'cssSelector',
    '다음으로 끝남': 'endsWith',
    '다음으로 끝나지 않음': 'endsWith',
}

negate_map = {
    '다음과 같지 않음': 'true',
    '같음': 'false',
    '_x0008_같음': 'false',
    '포함': 'false',
    '정규 표현식과 일치': 'false',
    '정규 표현식과 일치 ': 'false',
    '다음으로 시작 하지 않음': 'true',
    '다음으로 시작': 'false',
    'CSS 선택 도구와 일치': 'false',
    '다음으로 끝남': 'false',
    '다음으로 끝나지 않음': 'true',
}

def create_trigger(request_body):
    parent=workspace['path']

    try:
        if request_body['name'].lower() in trigger_obj:
            return
        response = service.accounts().containers().workspaces().triggers().create(
            parent=parent,
            body=request_body
        ).execute()

        print(f"Created : {response['name']}", end="")
        time.sleep(4)
        return response
    except Exception as e:
        print(request_body)
        print('\n' + f"An error occurred: {e}")
        print(f"Above error occurred with: {request_body['name']}")
        time.sleep(4)
        return e


In [50]:
def create_trigger_scroll(sheet):
    error = 0
    request_body = {}

    for index, row in sheet.iterrows():
        if type(row['트리거명']) == str:
            if (index != 0):
                try:
                    create_trigger(request_body)
                except Exception as e:
                    error += 1

            request_body = {
                "name": row['트리거명'],
                "type": event_type_map[row['트리거 유형']],
                "parameter": [
                    {
                        'type': 'template', 
                        'key': 'verticalThresholdUnits', 
                        'value': 'PERCENT'
                    }, 
                    {
                        'type': 'template', 
                        'key': 'verticalThresholdsPercent', 
                        'value': row['비율']
                    }, 
                    {
                        'type': 'boolean', 
                        'key': 'verticalThresholdOn', 
                        'value': 'true'
                    }, 
                    {
                        'type': 'template', 
                        'key': 'triggerStartOption', 
                        'value': 'WINDOW_LOAD'
                    }, 
                    {
                        'type': 'boolean', 
                        'key': 'horizontalThresholdOn', 
                        'value': 'false'
                    }
                ],
                "filter": []
            }


        request_body['filter'].append({
            "type": filter_type_map[row['연산자(메뉴)']],
            "parameter": [
                {
                    "type": "template",
                    "key": "arg0",
                    "value": "{{" + row['변수(메뉴)'] + "}}"
                },
                {
                    "type": "template",
                    "key": "arg1",
                    "value": row['값(텍스트 입력란)']
                },
                {
                    "type": "boolean",
                    "key": "negate",
                    "value": negate_map[row['연산자(메뉴)']]
                }
            ]
        })

    try:
        create_trigger(request_body)
    except Exception as e:
        error += 1

    print(f"Total Error: {error}")


In [51]:
def create_trigger_click(sheet):
    error = 0
    request_body = {}

    for index, row in sheet.iterrows():
        if type(row['트리거명']) == str:
            if (index != 0):
                try:
                    create_trigger(request_body)
                except Exception as e:
                    error += 1

            request_body = {
                "name": row['트리거명'],
                "type": event_type_map[row['트리거 유형']],
                'waitForTags': {'type': 'boolean', 'value': 'false'}, 
                'checkValidation': {'type': 'boolean', 'value': 'false'}, 
                'waitForTagsTimeout': {'type': 'template', 'value': '2000'}, 
                'uniqueTriggerId': {'type': 'template'},
                "filter": []
            }

        request_body['filter'].append({
            "type": filter_type_map[row['연산자(메뉴)']],
            "parameter": [
                {
                    "type": "template",
                    "key": "arg0",
                    "value": "{{" + row['변수(메뉴)'] + "}}"
                },
                {
                    "type": "template",
                    "key": "arg1",
                    "value": row['값(텍스트 입력란)']
                },
                {
                    "type": "boolean",
                    "key": "negate",
                    "value": negate_map[row['연산자(메뉴)']]
                }
            ]
        })

    try:
        create_trigger(request_body)
    except Exception as e:
        error += 1

    print(f"Total Error: {error}")


In [52]:
def create_trigger_page_view(sheet):
    error = 0
    request_body = {
        'filter': [],
    }

    for index, row in sheet.iterrows():
        if type(row['트리거명']) == str:
            if (index != 0):
                try:
                    create_trigger(request_body)
                except Exception as e:
                    error += 1

            request_body = {
                "name": row['트리거명'],
                "type": event_type_map[row['트리거 유형']],
                "filter": []
            }
        
        if type(row['연산자(메뉴)']) != str:
            continue

        request_body['filter'].append({
            "type": filter_type_map[row['연산자(메뉴)']],
            "parameter": [
                {
                    "type": "template",
                    "key": "arg0",
                    "value": "{{" + row['변수(메뉴)'] + "}}"
                },
                {
                    "type": "template",
                    "key": "arg1",
                    "value": row['값(텍스트 입력란)']
                },
                {
                    "type": "boolean",
                    "key": "negate",
                    "value": negate_map[row['연산자(메뉴)']]
                }
            ]
        })

    try:
        create_trigger(request_body)
    except Exception as e:
        error += 1

    print(f"Total Error: {error}")


In [53]:
def create_event_custom(sheet):
    error = 0
    for index, row in sheet.iterrows():
        if type(row['트리거명']) == str:
            if (index != 0):
                try:
                    create_trigger(request_body)
                except Exception as e:
                    error += 1

            request_body = {
                "name": row['트리거명'],
                "type": event_type_map[row['트리거 유형']],
                "customEventFilter": [
                    {
                        "type": "equals",
                        "parameter": [
                            {
                                "type": "template",
                                "key": "arg0",
                                "value": "{{_event}}"
                            },
                            {
                                "type": "template",
                                "key": "arg1",
                                "value": row['이벤트 이름']
                            }
                        ]
                    }
                ],
                "filter": []
            }

        if type(row['변수(메뉴)']) == str:
            request_body['filter'].append({
                "type": filter_type_map[row['연산자(메뉴)']],
                "parameter": [
                    {
                        "type": "template",
                        "key": "arg0",
                        "value": "{{" + row['변수(메뉴)'] + "}}"
                    },
                    {
                        "type": "template",
                        "key": "arg1",
                        "value": row['값(텍스트 입력란)']
                    },
                    {
                        "type": "boolean",
                        "key": "negate",
                        "value": negate_map[row['연산자(메뉴)']]
                    }
                ]
            })

    try:
        create_trigger(request_body)
    except Exception as e:
        error += 1

    print(f"Total Error: {error}")


In [54]:
sheet_names = get_sheet_name(xlsx)

for index, i in enumerate(sheet_names):
    print(index, i)

0 변수(상수)
1 변수(데이터 영역)
2 변수(DOM 요소)
3 변수(참고표)
4 변수(자동 이벤트 변수)
5 변수(이벤트 매개변수)
6 트리거(맞춤 이벤트)
7 트리거(링크 클릭)
8 트리거(페이지뷰)
9  태그(GA4)
10  태그(Firebase)


In [55]:
sheet_custom_event = read_sheet(xlsx, sheet_names[6])
sheet_page_view = read_sheet(xlsx, sheet_names[8])
# sheet_scroll = read_sheet(xlsx, sheet_names[10])
sheet_link = read_sheet(xlsx, sheet_names[7])
# sheet_link_2 = read_sheet(xlsx, sheet_names[8])
# sheet_click = read_sheet(xlsx, sheet_names[13])
# sheet_click_2 = read_sheet(xlsx, sheet_names[14])

create_event_custom(sheet_custom_event)
create_trigger_page_view(sheet_page_view)
# create_trigger_scroll(sheet_scroll)
create_trigger_click(sheet_link)
# create_trigger_click(sheet_link_2)
# create_trigger_click(sheet_click)
# create_trigger_click(sheet_click_2)

Total Error: 0
Total Error: 0
Total Error: 0
